In [ ]:
import pandas as pd
import re
input1= pd.read_csv("C:\\Users\\Alex\\Desktop\\Python\\CI_Analyst_-_input1.csv")
input2= pd.read_csv("C:\\Users\\Alex\\Desktop\\Python\\CI_Analyst_-_input2.csv")
#create age, cluster, device columns
input1['campaign_name']=input1['campaign_name'].str.lower().str.strip() 
input1_unfold=pd.concat([input1,pd.DataFrame(input1["campaign_name"].str.split('_',n=2).tolist())],axis=1,join='outer')
input1_unfold.rename(columns={0:'age', 1 :'cluster',2: 'device'},inplace=True)
#creates key by removing date
input1_unfold['key']=input1_unfold["campaign_name"].str.rstrip('_1234567890')
#creates device by removing date
input1_unfold['device']=input1_unfold["device"].str.rstrip('_1234567890')
from ast import literal_eval #import literal eval
input2['actions']=input2['actions'].replace("'","\"") #replace ' with "
input2['actions']=input2['actions'].apply(literal_eval) #apply literal_eval function on action
my_list=['share','comment','like'] #define what is considered special engagement
n=len(input2['actions']) #find number of actions
se_count=[] #create empty list
for i in range(0,n): #for loop for all actions
    count=0 #set counter to 0
    for index,dict in enumerate(z14['actions'][i]): #for loop each action entry
        if any(type in dict['action_type'] for type in my_list): #sees if special engagement is a part of that action
            count+= dict['value'] #adds the value if the special engagement is
    se_count.append(count)
input2['storied_engagements']=se_count
#convert the key to lower and strip whitespaces, data cleaning
input1_unfold['key']=input1_unfold['key'].str.lower().str.strip()
input2['campaign_name']=input2['campaign_name'].str.lower().str.strip()
#inner merge
merged=input1_unfold.merge(input2,left_on='key',right_on='campaign_name')
len(merged) #there were only 174 matches.
n2=len(merged['object_type']) #counts number of rows
CPVV_list=[] #create empty list to store CPVV values

for i in range(0,n2): #for loop for all object_types
    if merged['object_type'][i]=="VIDEO": #checks object type for video
        CPVV_view_options=[] #empty list to store local CPVV values, noticed that some object_type== VIDEO didnt not have video views
        for index,dict in enumerate(merged['actions'][i]): #for loop each action entry
            if dict['action_type']=='video_view': #finds where action_type == video_view
                s= merged['spend'][i] #gets spend for campaign
                v=dict['value'] #gets views for campaign
                CPVV_view=s/v #calculates CPVV, spend/video
                CPVV_view_options.append(CPVV_view) #adds to list
            else: #when the action type is not video_views
                CPVV_notviewed=0
                CPVV_view_options.append(CPVV_notviewed)
            CPVV_video=max(CPVV_view_options) #use max to extract the CPVV value that is relevant
        CPVV_list.append(CPVV_video) #appends to global list
    else: #for object types that are not video
        CPVV_notvideo=0
        CPVV_list.append(CPVV_notvideo)
merged['CPVV']=CPVV_list #creates CPVV column in dataframe
#aggregate functions for group by
functions={'spend':['sum'],'impressions':['sum'],'storied_engagements':['sum'],'campaign_name_x':['nunique'],'CPVV':['sum']}
#group by based on columns
grouped_key=merged.groupby('key').agg(functions)
grouped_age=merged.groupby('age').agg(functions)
grouped_cluster=merged.groupby('cluster').agg(functions)
grouped_device=merged.groupby('device').agg(functions)
#sorting the dataframe
sorted_key=merged.sort_values("key")
sorted_age=merged.sort_values('age')
sorted_cluster=merged.sort_values('cluster')
sorted_device=merged.sort_values('device')
#function to save multiple dfs to same sheet, and multiple sheets
def dfs2xlsx(list_dfs,filename,list_sheetnames= [],directory = 'C:\\Users\\Alex\\Desktop\\Python\\'):
    """list_dfs is each set of df on one sheet.sheetname is the name give in excel to those sheets"""
  
    if list_sheetnames==[]: #if no sheetnames are given, default are indexes of list_dfs
        sheetnames=[i+1 for i,x in enumerate(list_dfs)]
    else:
        sheetnames=list_sheetnames #if sheetnames given, then take those
    xls_path=directory+filename+".xlsx" #creates xlspath from directory and filename
    writer = pd.ExcelWriter(xls_path,engine='xlsxwriter') #create writer
    for a, sheet in enumerate(list_dfs): #each list in list_df is a sheet
        i=0 #start a column 1
        for b, df in enumerate(list_dfs[a]): #access the dataframes within each list of list_df
            df.to_excel(writer,sheet_name="'"+sheetnames[a]+"'",startcol=i,startrow =2)
            i+= len(df.columns) + 2 #spaces over
    writer.save()
list_tables=[[grouped_key,sorted_key],[grouped_age,sorted_age],[grouped_cluster,sorted_cluster],[grouped_device,sorted_device]] #dfs to be exported and their pairing
list_names=['key','age','cluster','device'] #sheetnames
dfs2xlsx(list_dfs=list_tables,filename='social_code_alex_tranv3',list_sheetnames=list_names) #exports to excel, not sure why it says file needs to be repaired